# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bermeo,43.4209,-2.7215,9.93,71,97,6.60,ES,1675641600
1,1,tuktoyaktuk,69.4541,-133.0374,-38.59,66,20,2.61,CA,1675641600
2,2,khatanga,71.9667,102.5000,-36.90,100,55,3.00,RU,1675641600
3,3,punta arenas,-53.1500,-70.9167,13.06,47,40,5.95,CL,1675641600
4,4,kaitangata,-46.2817,169.8464,19.24,58,69,8.05,NZ,1675641600


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df [city_data_df["Max Temp"]<27]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bermeo,43.4209,-2.7215,9.93,71,97,6.60,ES,1675641600
1,1,tuktoyaktuk,69.4541,-133.0374,-38.59,66,20,2.61,CA,1675641600
2,2,khatanga,71.9667,102.5000,-36.90,100,55,3.00,RU,1675641600
3,3,punta arenas,-53.1500,-70.9167,13.06,47,40,5.95,CL,1675641600
4,4,kaitangata,-46.2817,169.8464,19.24,58,69,8.05,NZ,1675641600
...,...,...,...,...,...,...,...,...,...,...
542,542,belgrade,44.8040,20.4651,-2.93,79,0,5.08,RS,1675641600
543,543,myitkyina,25.3833,97.4000,13.35,68,28,1.17,MM,1675641600
544,544,khovu-aksy,51.1333,93.6000,-16.32,98,98,0.42,RU,1675641600
546,546,korla,41.7597,86.1469,-3.03,71,3,1.31,CN,1675641600


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bermeo,ES,43.4209,-2.7215,71,
1,tuktoyaktuk,CA,69.4541,-133.0374,66,
2,khatanga,RU,71.9667,102.5000,100,
3,punta arenas,CL,-53.1500,-70.9167,47,
4,kaitangata,NZ,-46.2817,169.8464,58,
...,...,...,...,...,...,...
545,oranjemund,NaN,-28.5500,16.4333,94,
546,korla,CN,41.7597,86.1469,71,
547,lufilufi,WS,-13.8745,-171.5986,76,
548,berlin,DE,52.5244,13.4105,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat=row["Lat"]
    Lng=row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"

    #params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
      
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
       
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bermeo - nearest hotel: Castillo de Arteaga
tuktoyaktuk - nearest hotel: Hunter's B&B
khatanga - nearest hotel: Хатанга
punta arenas - nearest hotel: Ainil
kaitangata - nearest hotel: Hotel South Otago
kourou - nearest hotel: Mercure Kourou Ariatel
barmer - nearest hotel: No hotel found
sorland - nearest hotel: Fiskarheimen
rikitea - nearest hotel: Chez Bianca & Benoit
moshenskoye - nearest hotel: No hotel found
arraial do cabo - nearest hotel: Pousada Vida Boa Praia
kavieng - nearest hotel: Kavieng Niu Lodge
san cristobal - nearest hotel: Nevada Suites
vaini - nearest hotel: Emerald Hotel
busselton - nearest hotel: Abbey Beach Resort
ushuaia - nearest hotel: Arakur
victoria - nearest hotel: 唯港薈 Hotel ICON
ribeira grande - nearest hotel: Residencial São Francisco
san esteban - nearest hotel: No hotel found
castro - nearest hotel: Hotel Fazenda das 100 Árvores
dikson - nearest hotel: No hotel found
twin falls - nearest hotel: No hotel found
peniche - nearest hotel:

puerto escondido - nearest hotel: Hotel Paraiso Escondido
esperance - nearest hotel: Hospitality
brightwater - nearest hotel: No hotel found
awbari - nearest hotel: فندق اوباري
tessalit - nearest hotel: No hotel found
miquelon - nearest hotel: No hotel found
pisco - nearest hotel: Gran Hotel Chincha
sant feliu de guixols - nearest hotel: Rosa dels Vents
mackay - nearest hotel: Mackay Oceanside Central Apartment Hotel
paamiut - nearest hotel: Hotel Ivaana
matara - nearest hotel: Amaloh by Jetwing
capreol - nearest hotel: No hotel found
lasa - nearest hotel: Gasthof Neue Post
oranjestad - nearest hotel: Hyatt Regency Aruba Resort Spa and Casino
dauphin - nearest hotel: No hotel found
mao - nearest hotel: Artiem Capri
mattru - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
houma - nearest hotel: Microtel Inn & Suites by Wyndham Houma
ostersund - nearest hotel: Hotell Östersund
ano mera - nearest hotel: Markos Beach Hotel
cidreira - nearest hotel: Lagoa Country Club
chug

abu samrah - nearest hotel: No hotel found
liusha - nearest hotel: No hotel found
noyabrsk - nearest hotel: Белые ночи
bethel - nearest hotel: Hotel Zero Degrees
port blair - nearest hotel: Hornbill Nest resort
luderitz - nearest hotel: Nest Hotel
port moresby - nearest hotel: Hilton Port Moresby
laguna - nearest hotel: TownePlace Suites Sacramento Elk Grove
riyadh - nearest hotel: Hilton Garden Inn Riyadh Olaya
constitucion - nearest hotel: Hotel Casona del Cerro
athabasca - nearest hotel: Days Inn
sibolga - nearest hotel: Hotel Hotma
eniwa - nearest hotel: ANAクラウンプラザ Chitose
caravelas - nearest hotel: Pousada dos Navegantes
sarh - nearest hotel: فندق سفاري
adre - nearest hotel: No hotel found
shache - nearest hotel: 莎車新盛賓舘
balgazyn - nearest hotel: No hotel found
julio de castilhos - nearest hotel: No hotel found
leningradskiy - nearest hotel: No hotel found
novyye lyady - nearest hotel: No hotel found
mizan teferi - nearest hotel: ሆቴል ሳላይሽ / Salayesh Hotel
kismayo - nearest hotel: ف

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bermeo,ES,43.4209,-2.7215,71,Castillo de Arteaga
1,tuktoyaktuk,CA,69.4541,-133.0374,66,Hunter's B&B
2,khatanga,RU,71.9667,102.5000,100,Хатанга
3,punta arenas,CL,-53.1500,-70.9167,47,Ainil
4,kaitangata,NZ,-46.2817,169.8464,58,Hotel South Otago
...,...,...,...,...,...,...
545,oranjemund,NaN,-28.5500,16.4333,94,Op My Stoep
546,korla,CN,41.7597,86.1469,71,Silver Star Hotel
547,lufilufi,WS,-13.8745,-171.5986,76,No hotel found
548,berlin,DE,52.5244,13.4105,79,Intercity Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = ["Country","Hotel Name"],
    hover = True
    
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)